<center>
<a href="https://exed.polytechnique.edu/fr" ><img src="https://exed.polytechnique.edu/sites/all/themes/college/images/logo.png" style="float:left; max-width: 360px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="float:right; max-width: 250px; display: inline"  alt="Wikistat"/></a>

</center>

# [Scénarios d'Apprentissage Statistique](https://github.com/wikistat/Apprentissage)

# GRC: Score d'appétence d'un produit bancaire  en <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a> avec <a href="http://scikit-learn.org/stable/#"><img src="http://scikit-learn.org/stable/_static/scikit-learn-logo-small.png" style="max-width: 100px; display: inline" alt="Scikit-learn"/></a>

#### Résumé
Les données sont composées de 825 clients d'une banque décrits par 32 variables concernant leurs avoirs, et utilisations de leurs comptes. Après avoir réalisé, avec [R](https://github.com/wikistat/Exploration/blob/master/GRC-carte_Visa/Explo-R-Visa.ipynb) ou [Python](https://github.com/wikistat/Exploration/blob/master/GRC-carte_Visa/Explo-Python-Visa.ipynb), le premier objectif de segmentation ou profilage des types de comportement des clients, le 2ème consiste à estimer puis prévoir un *score d'appétence* pour un produit bancaie, ici la carte visa premier. Comparaison des différentes méthodes et algorihtmes d'apprentissage pour atteindre cet objectif de la régression logistique au *boosting* (*extrem gradient*) en passant par les arbres, les SVM ou random forest. Une procédure de validation croisée généralisée est itérée sur une selection de ces méthodes. Celles d'agrégation de modèles conduisent aux meilleurs résultats. 

## Introduction
### Objectif
Un  [calepin]((https://github.com/wikistat/Exploration/blob/master/GRC-carte_Visa/Explo-Python-Visa.ipynb), qu'il est préférable d'exécuter préalablement, décrit le premier objectif d'exploration puis segmentation ou profilage des types de comportement des clients d'une banque. 

Ce deuxième calepin propose de construire un [score d'appétence](http://www.math.univ-toulouse.fr/~besse/Wikistat/pdf/st-scenar-app-visa.pdf) pour la carte *Visa Premier*. Il s'agit d'un score d'appétence mais ce pourrait être le score d'attrition (*churn*) d'un opérateur téléphonique ou encore un score de défaillance d'un emprunteur ou de faillite d'une entreprise; les outils de modélisation sont les mêmes et sont très largement utilisés dans tout le secteur tertiaire pour l'aide à la décision.

### Présentation des données
#### Les variables
La liste des variables est issue d'une base de données retraçant l'historique mensuel bancaire et les caractéristiques de tous les clients. Un sondage a été réalisé afin d'alléger les traitements ainsi qu'une première sélection de variables. Les variables contenues dans le fichier initial sont décrites dans le tableau ci-dessous. Elles sont observées sur 1425 clients.

*Tableau: Liste des variables initiales et de leur libellé* Attention, certains sont écrits en majuscules dans les programmes puis en minuscules après transfomation des données (logarithme, recodage) au cours d ela phase d'exploration. Les noms des variables logarithmes des variables quantitatives se terminent par `L`les variables qualitatives se terminent par `Q`ou `q`. 

**Identifiant** | **Libellé**
           --|--
`sexeq` | Sexe (qualitatif) 
`ager` | Age en années
`famiq` | Situation familiale: `Fmar Fcel Fdiv Fuli Fsep Fveu`
`relat` | Ancienneté de relation en mois
`pcspq` | Catégorie socio-professionnelle (code num)
`opgnb` | Nombre d'opérations par guichet dans le mois
`moyrv` | Moyenne des mouvements nets créditeurs des 3 mois en Kf
`tavep` | Total des avoirs épargne monétaire en francs
`endet` | Taux d'endettement
`gaget` | Total des engagements en francs
`gagec` | Total des engagements court terme en francs
`gagem` | Total des engagements moyen terme en francs
`kvunb` | Nombre de comptes à vue
`qsmoy` | Moyenne des soldes moyens sur 3 mois
`qcred` | Moyenne des mouvements créditeurs en Kf
`dmvtp` | Age du dernier mouvement (en jours)\hline
`boppn` | Nombre d'opérations à M-1
`facan` | Montant facturé dans l'année en francs
`lgagt` | Engagement long terme
`vienb` | Nombre de produits contrats vie
`viemt` | Montant des produits contrats vie en francs
`uemnb` | Nombre de produits épargne monétaire
`xlgnb` | Nombre de produits d'épargne logement
`xlgmt` | Montant des produits d'épargne logement en francs
`ylvnb` | Nombre de comptes sur livret
`ylvmt` | Montant des comptes sur livret en francs
`rocnb` | Nombre de paiements par carte bancaire à M-1
`nptag` | Nombre de cartes point argent
`itavc` | Total des avoirs sur tous les comptes
`havef` | Total des avoirs épargne financière en francs
`jnbjd | Nombre de jours à débit à M
**`carvp`** | **Possession de la carte VISA Premier**


**Répondre aux questions en s'aidant des résultats des exécutions**

## Préparation des données
### Lecture 
Les données sont disponibles dans le répertoire de ce calepin et chargées en même temps. Elles sont issues de la première phase de [prétraitement](https://github.com/wikistat/Exploration/blob/master/GRC-carte_Visa/Explo-R-Visa.ipynb) ou *data munging* pour détecter, corriger les erreurs et incohérences, éliminer des redondances, traiter les données manquantes, transformer certaines variables. 

In [ ]:
# Importation des librairies.
import numpy as np
import pandas as pd
import random as rd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
npt_config = {
    'session_name': 'Visa-ML',
    'session_owner': 'aymeric',
    'sender_name': input("Your name:"),
} 

In [ ]:
# Lecture d'un data frame en ligne
! wget -q https://raw.githubusercontent.com/adieulev/DSO/master/vispremv.dat
vispremv = pd.read_table('vispremv.dat', delimiter=' ')

#Donner la taille des données

In [ ]:
# Lecture d'un data frame si vous travaillez en local 
# Placer la table dans le bon dossier et décommenter la ligne suivante
# vispremv = pd.read_table('vispremv.dat', delimiter=' '

In [ ]:
vispremv.head()

In [ ]:
# Isoler les Variables quantitatives

##TODO

Vérifier ci-dessous que la plupart des variables ont deux versions, l'une quantitative et l'autre qualitative.

Les variables qualitatives (sexe, csp, famille) sont transformées en indicatrices à l'exception de la cible `CARVP`.

In [ ]:
vispremv.dtypes

In [ ]:
# Transformation en indicatrices des variables qualitatives
# Utiliser pd.get_dummies


#TODO

# Concaténation ensuite avec les variables numériques
# pd.concat([vispremDum,vispremNum],axis=1)

#TODO

send('Dummy vairable done', 0)

**Q** Combien d'individus et combien de variables sont finalement concernés? 

In [ ]:
#TODO calculer nombre individus et variables

send(np.array(vispremR.shape), 1)

In [ ]:
# La variable à expliquer est recodée
y=vispremv["CARVP"].map(lambda x: 0 if x=="Cnon" else 1)

### Extraction des échantillons apprentissage et test

In [ ]:
rd_seed=111 # Modifier cette valeur d'initialisation
npop=len(vispremv)
#TODO trouver xApp,xTest,yApp,yTest a laide de train_test_split
#On placera 200 points dans l'ensmeble de test

send(np.array(xApp.shape), 2)

## [Régression logistique](http://wikistat.fr/pdf/st-m-app-rlogit.pdf)
Cette ancienne méthode reste toujours très utilisée. D'abord par habitude mais aussi par efficacité pour le traitement de données très volumineuses lors de l'estimation de très gros modèles (beaucoup de variables) notamment par exemple chez Criteo ou CDiscount.

### Estimation et optimisation
La procédure de sélection de modèle est celle par pénalisation: *ridge*, Lasso ou une combinaison (*elastic net*). Contrairement aux procédures disponibles en R (*stepwise, backward, forward*) optimisant un critère comme *AIC*, l'algorithme proposé dans `scikit-learn` nepermet pas une prise en compte simple des interactions. D'autre part les compléments usuels (test de Wald ou du rapport de vraisemblance) ne sont pas directement fournis. 



#### Remarque importante: 

POur la cross validation, doit on couper le **dataset complet** en train + validation sets ou bien seulement **l'ensemble d'apprentissage**?

A première vue, on pourrait donner le dataset entier, car on va déjà dans la cross validation couper en deux, mais en fait il faut couper en trois:
- un test set qu'on réserve 
- un ensemble d'apprentissage sur lequel on fait de la validation croisée (donc qu'on va couper de nombreuses fois en un train set + un validation set)

Sinon, comme on va choisir le paramètre qui a le meilleur résultat sur l'ensemble de validation (ou qui a le meilleur résultat après cross validation), il est possible qu'on choisisse un paramètre pour lequel le score était particuliérement bon "par chance".

On évaluera toujours en dernier lieu la performance sur un dataset (le test) complètement indépendant du processus de sélection des paramètres (le train) ou hyperparamètres (le validation).

C'est ce qui apparaissait dans le pipeline au premier cours !

<center>
<img src="http://www.cmap.polytechnique.fr/~aymeric.dieuleveut/papers/This-is-ML-pipe" style="float:left; max-width: 600px; display: inline" alt="INSA"/></center>
<br>


Voir par exemple: https://towardsdatascience.com/train-validation-and-test-sets-72cb40cba9e7


#### Optimisation *ridge*
On considère maintenant l'optimisation Ridge, ou la pénalité est proportionnelle à la norme 2 de l'estimateur (au carré)


In [ ]:
# Grilles de valeurs du paramètre de pénalisation
param=[{"C":[0.5,1,5,10,12,15,30]}]
# TODO
# 1. Definir le modèle (regression logistique)
# 2. On utilisera ici une pénalité l1 penalty="l2"
# 3. Utiliser GridSearchCV pour obtenir directement le meilleur paramètre de régularisation
# 4. On appellera la méthode logitRidge pour la suite ! 

In [ ]:
# Calcul erreur
print("Meilleur score = %f, Meilleur paramètre = %s" %
      (1. - logitRidge.best_score_, logitRidge.best_params_))
send(("Meilleur score = %f, Meilleur paramètre = %s" % (1. - logitRidge.best_score_, logitRidge.best_params_)), 3)

In [ ]:
# Prévision
yChap = logitRidge.predict(xTest)

#Todo matrice de confusion


# Erreur sur l'échantillon test
print("Erreur de test régression Ridge = %f" % (1-logitRidge.score(xTest, yTest)))

**Q** Noter l'erreur de prévision; Comparer avec celle estimée par validation croisée.

### Interprétation

L'objet logitRidge issu de GridSearchCV ne retient pas les paramètres estimés dans le modèle. Il faut donc ré-estimer avec la valeur optimale du paramètre de pénalisation si l'on souhaite afficher ces coefficients.

In [ ]:
RidgeOpt=LogisticRegression(penalty="l2",C=12) # replace by the best parameters !!
RidgeOpt=LassoOpt.fit(xApp, yApp)
# Récupération des coefficients
vect_coef=np.matrix.transpose(LassoOpt.coef_)
vect_coef=vect_coef.ravel()
#Affichage des 25 plus importants
coef=pd.Series(abs(vect_coef),index=xApp.columns).sort_values(ascending=False)
print(coef)

In [ ]:
plt.figure(figsize=(7,4))
coef.plot(kind='bar')
plt.title('Coeffients')
plt.tight_layout()
send(plt, 4)
plt.show()

**Q** Quelles sont les variables importantes? Comment interpréter?

**Q** La pénalisation Lasso est-elle effective?

Il serait intéressant de comparer acec les versions *ridge* et *elestic net* d'optiisation du modèle.

### Courbe ROC

In [ ]:
from sklearn.metrics import roc_curve
listMethod=[["Lasso",logitLasso],["Ridge",logitRidge]]

for method in enumerate(listMethod):
    probas_ = method[1][1].predict_proba(xTest)
    fpr, tpr, thresholds = roc_curve(yTest, probas_[:,1])
    plt.plot(fpr, tpr, lw=1,label="%s"%method[1][0])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.legend(loc="best")
send(plt, 5)
plt.show()

## Analyse discriminante
Trois méthodes sont disponibles: paramétrique linéaire ou quadratique et non paramétrique (*k* plus proches voisins).

In [ ]:
from sklearn import discriminant_analysis
from sklearn.neighbors import KNeighborsClassifier

### Dicriminante linéaire
Estimation du modèle; il n'y a pas de procédure de sélection de variables proposées. Puis prévision de l'échantillon test.

In [ ]:
# TODO:
# 1. definir la methode 'lda' comme discriminant_analysis.LinearDiscriminantAnalysis

# 2. fitter la méthode

# 3. Prédire sur  l'échantillon test

# 4. Calculer la matrice de confusion


# Erreur de prévision sur le test
print("Erreur de test lda = %f" % (1-disLin.score(xTest,yTest)))
send(("Erreur de test lda = %f" % (1-disLin.score(xTest,yTest))), 6)

**Q** Que dire de la qualité? Des possibilités d'interprétation?

**Q** Que signifie le *warning*? Quelles variables osnt en cause?
### Discriminante quadratique

In [ ]:
# Procéder de même pour QDA

### K plus proches voisins

In [ ]:

# 1. Définition du modèle avec 10 plus proches voisins (utiliser KNeighborsClassifier)

# 2. Fitter
# 3. Prévision de l'échantillon test

# 4. matrice de confusion

print(table)
# Erreur de prévision sur le test
print("Erreur de test knn = %f" % (1-disKnn.score(xTest,yTest)))
s

In [ ]:
#Optimisation du paramètre de complexité (nombre de voisins) k
#Grille de valeurs
param_grid=[{"n_neighbors":list(range(1,15))}]
disKnn=GridSearchCV(KNeighborsClassifier(),param_grid,cv=5,n_jobs=-1)
disKnnOpt=disKnn.fit(xApp, yApp) # GridSearchCV est lui même un estimateur
# paramètre optimal
disKnnOpt.best_params_["n_neighbors"]
print("Meilleur score = %f, Meilleur paramètre = %s" % (1.-disKnnOpt.best_score_,disKnnOpt.best_params_))
send(("Meilleur score = %f, Meilleur paramètre = %s" % (1.-disKnnOpt.best_score_,disKnnOpt.best_params_)), 7)

In [ ]:
# Prévision de l'échantillon test
yChap = disKnnOpt.predict(xTest)
# matrice de confusion
table=pd.crosstab(yChap,yTest)
print(table)

# Estimation de l'erreur de prévision sur l'échantillon test
print("Erreur de test knn_opt = %f" % (1-disKnnOpt.score(xTest,yTest)))

Courbes ROC

In [ ]:
from sklearn.metrics import roc_curve
# Liste des méthodes
listMethod=[["lda",disLin],["qda",disQua],["knn",disKnnOpt]]
# Tracé des courbes
for method in enumerate(listMethod):
    probas_ = method[1][1].predict_proba(xTest)
    fpr, tpr, thresholds = roc_curve(yTest, probas_[:,1])
    plt.plot(fpr, tpr, lw=1,label="%s"%method[1][0])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.legend(loc="best")
plt.show()

## [Arbres binaires de décision](http://wikistat.fr/pdf/st-m-app-cart.pdf)
Les arbres binaires de décision concurrencent la régression logistique et gardent une place de choix dans les services de Gestion de la Relation Client, maintenant de *Science des Données*, par la facilité d'interprétation des modèles qui en découlent. L'optimisation de la complexité d'un artbre peut être délicate à opérer cr très sensible aux fluctuations de l'échantillon.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# définition et fit du modèle
# TODO

**Q** Quel est le critère d'homogénéité des noeuds utilisé par défaut?

**Q** Quel est le problème concernant l'élagage de l'arbre dans `Scikkit-learn` vis à vis des possibliités de la librairie `rpart` de R?

In [ ]:
# Optimisation de la profondeur de l'arbre
#TODO

# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - treeOpt.best_score_,treeOpt.best_params_))
send(("Meilleur score = %f, Meilleur paramètre = %s" % (1. - treeOpt.best_score_,treeOpt.best_params_)), 8)

In [ ]:
# Prévision de l'échantillon test
yChap = treeOpt.predict(xTest)
# matrice de confusion
table=pd.crosstab(yChap,yTest)
print(table)# Erreur de prévision sur le test
print("Erreur de test tree qualitatif = %f" % (1-treeOpt.score(xTest,yTest)))

In [ ]:
# Visualisation
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
import pydotplus
treeG=DecisionTreeClassifier(max_depth=treeOpt.best_params_['max_depth'])
treeG.fit(xApp,yApp)
dot_data = StringIO() 
export_graphviz(treeG, out_file=dot_data) 
graph=pydotplus.graph_from_dot_data(dot_data.getvalue()) 
graph.write_png("treeOpt.png")  

In [ ]:
from IPython.display import Image
Image(filename='treeOpt.png')

### [Courbes ROC](http://wikistat.fr/pdf/st-m-app-risque.pdf)
Comparaison des méthodes précédentes.

La valeur de seuil par défaut (0.5) n'étant pas nécessairement celle "optimale", il est important de comparer les courbes ROC.

In [ ]:
# Liste des méthodes
listMethod=[["Logit",logitLasso],["lda",disLin],["Arbre",treeOpt]]
# Tracé des courbes
for method in enumerate(listMethod):
    probas_ = method[1][1].predict_proba(xTest)
    fpr, tpr, thresholds = roc_curve(yTest, probas_[:,1])
    plt.plot(fpr, tpr, lw=1,label="%s"%method[1][0])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.legend(loc="best")
send(plt, 9)
plt.show()

Commenter les résultats.

**Q** Intérêt de la régression logistique par rapport à l'analyse discriminante linéaire?

**Q** Conséquence du croisement des courbes ROC sur l'évaluation de l'AUC.

L'échantillon test reste de taille modeste (200). une étude plus systématique est nécessaire ainsi que la prise en compte des autres méthodes.

## [Algorithmes d'agrégation de modèles](http://wikistat.fr/pdf/st-m-app-agreg.pdf)
Il s'agit de comparer les principaux algorithmes issus de l'apprentissage machine: *bagging, random forest, boosting*.

### *Bagging*

**Q** Quel est par défaut l'estimateur qui est agrégé? 

**Q** Quel est le nombre d'estimateurs par défaut? Est-il nécessaire de l'optimiser?

In [ ]:
from sklearn.ensemble import BaggingClassifier
bag= # Def model (nestim =100)
bagC=bag.fit(xApp, yApp)
# Prévision de l'échantillon test
yChap = bagC.predict(xTest)
# matrice de confusion
table=pd.crosstab(yChap,yTest)
print(table)

# Erreur de prévision sur le test
print("Erreur de test avec le bagging = %f" % (1-bagC.score(xTest,yTest)))
send(("Erreur de test avec le bagging = %f" % (1-bagC.score(xTest,yTest)), 10))

**Q** Exécuter plusieurs fois la cellule ci-dessus. Que penser de la stabilité de l'estimation de l'erreur et donc de sa fiabilité?

### *Random forest*

**Q** Quel paramètre doit être optimisé pour cet algorithme? Quel est sa valeur par défaut?

**Q** Le nombre d'arbres de la forêt est-il un paramètre sensible?

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Optimisation de max_features
param=[{"max_features":list(range(2,10,1))}]

rf= # TODO DEF gridsearchCV
rfOpt=rf.fit(xApp, yApp)
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - rfOpt.best_score_,rfOpt.best_params_))
send(("Meilleur score = %f, Meilleur paramètre = %s" % (1. - rfOpt.best_score_,rfOpt.best_params_)), 11)

In [ ]:
# Prévision de l'échantillon test
yChap = rfOpt.predict(xTest)
# matrice de confusion
table=pd.crosstab(yChap,yTest)
print(table)

# Erreur de prévision sur le test
print("Erreur de test random forest opt -quantitatif = %f" % (1-rfOpt.score(xTest,yTest)))

### *Gradient boosting*

**Q** Quel est l'algorithme de *boosting* historique? Lequel est utilisé ici?

**Q** Quels sont les paramètres qu'il est important de contrôler, optimiser?

**Q** Quelle est la valeur par défaut de celui non optimisé ci-dessous?

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Optimisation de deux paramètres
paramGrid = [
  {'n_estimators': list(range(100,601,50)), 'learning_rate': [0.1,0.2,0.3,0.4]}
 ]
gbmC= GridSearchCV(GradientBoostingClassifier(),paramGrid,cv=5,n_jobs=-1)
gbmOpt=gbmC.fit(xApp, yApp)
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (1. - gbmOpt.best_score_,gbmOpt.best_params_))

In [ ]:
# Prévision de l'échantillon test
yChap = gbmOpt.predict(xTest)
# matrice de confusion
table=pd.crosstab(yChap,yTest)
print(table)

# Erreur de prévision sur le test
print("Erreur de test gbm opt = %f" % (1-gbmOpt.score(xTest,yTest)))
send(("Erreur de test gbm opt = %f" % (1-gbmOpt.score(xTest,yTest))), 12)

### Courbes ROC

In [ ]:
# Liste des méthodes
listMethod=[["Logit",logitLasso],["lda",disLin],["Arbre",treeOpt],["RF",rfOpt],["GBM",gbmOpt]]
# Tracé des courbes
for method in enumerate(listMethod):
    probas_ = method[1][1].predict_proba(xTest)
    fpr, tpr, thresholds = roc_curve(yTest, probas_[:,1])
    plt.plot(fpr, tpr, lw=1,label="%s"%method[1][0])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.legend(loc="best")
send(plt, 13)
plt.show()

**Q** Quelles meilleure méthode interprétable? Quelle meilleure méthode?

**Q** Que dire de l'*extrem gradient boosting* ? Du nombre de paramètres à optimiser? De son implémentation en Python par rapport à R? De sa disponibilité sous Windows?

**Exercice** Ajouter les réseaux de neurones et les SVM dans la comparaison.

#### Optimisation Lasso

In [ ]:
from sklearn.linear_model import LogisticRegression
# Grille de valeurs du paramètre de pénalisaiton
param = [{"C": [0.5, 1, 5, 10, 12, 15, 30]}] # dictionnaire de valeurs de C, utilisé pour GridSearchCV
# TODO
# 1. Definir le modèle (regression logistique)
# 2. On utilisera ici une pénalité l1 penalty="l1"
# 3. Utiliser GridSearchCV pour obtenir directement le meilleur paramètre de régularisation
# 4. On appellera la méthode logitLasso pour la suite !






In [ ]:
# CAlculer perte
print("Meilleur score (apprentissage) = %f, Meilleur paramètre = %s" %
      (1.-logitLasso.best_score_,logitLasso.best_params_))
send("Meilleur score (apprentissage) = %f, Meilleur paramètre = %s" %
      (1.-logitLasso.best_score_,logitLasso.best_params_),5)

Erreur de prévision

In [ ]:
# Prévision
yChap = logitLasso.predict(xTest)
# TODO:  calculer la matrice de confusionen utilisant pd.crosstab

print(table)
print("Erreur de test régression Lasso = %f" % (1-logitLasso.score(xTest, yTest)))
send("Erreur de test régression Lasso = %f" % (1-logitLasso.score(xTest, yTest)), 14)